In [1]:
!pip install datasets transformers

In [2]:
import os 
from torchvision.datasets import ImageFolder

In [3]:
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.data import DataLoader

In [ ]:
import shutil
os.mkdir("trainBalanced")
folders = ["F","M","N","Q","S","V"]

for folder in folders :
    os.mkdir(f"trainBalanced/{folder}")
    images = os.listdir(f"{path}/{folder}")
    images = images[:642]
    for img in images:
        shutil.copyfile(f"{path}/{folder}/{img}",f"trainBalanced/{folder}/{img}")




In [36]:
from torch.utils.data import random_split
train_path = "/kaggle/input/ecg-image-data/ECG_Image_data/train"
test_path = "/kaggle/input/ecg-image-data/ECG_Image_data/test"

transform = Compose([
    Resize((224,224)),  # Resize the image
    ToTensor()           # Convert image to tensor
])


train_dataset = ImageFolder(
    root="./trainBalanced",
    transform = transform
    
)
total_size = len(train_dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

train_data, test_data = random_split(train_dataset, [train_size, test_size])

#test_data = ImageFolder(
#    root=test_path,
#    transform = transform
#)

In [5]:
from transformers import ViTImageProcessor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name_or_path)


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [6]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00


In [7]:
import torch
import numpy as np
import evaluate


def collate_fn(batch):
    return {
        'pixel_values': torch.stack([image for image,_ in batch]),
        'labels': torch.tensor([label for _,label in batch])
    }



metric = evaluate.load("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


In [11]:
from transformers import ViTForImageClassification

labels = test_data.classes
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-beans",
  per_device_train_batch_size=32,
  evaluation_strategy="epoch",  # Evaluate once per epoch
  num_train_epochs=1,           # Train for exactly 2 epochs
  save_strategy="epoch",
  fp16=True,                    # Use mixed precision for faster training
  logging_dir="./logs",         # Directory for TensorBoard logs
  logging_steps=500,            # Log every 500 steps (optional adjustment)
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to="tensorboard",
  load_best_model_at_end=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [38]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=processor,
)



<ipython-input-38-72dc0e36a0e4>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [39]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.000000,1.000000


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


***** train metrics *****
  epoch                    =         1.0
  total_flos               = 222363862GF
  train_loss               =         0.0
  train_runtime            =  0:01:31.76
  train_samples_per_second =      33.574
  train_steps_per_second   =       0.534


In [40]:
test_data = ImageFolder(
    root=test_path,
    transform = transform
)

In [41]:
metrics = trainer.evaluate(test_data)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =        1.0
  eval_loss               =        0.0
  eval_runtime            = 0:05:06.34
  eval_samples_per_second =     80.951
  eval_steps_per_second   =       5.06


In [42]:
len(train_data)

3081